In [2]:
"""
Create all figures for Chapter 2: Theoretical Background
Now with Binary-lens caustics (VBBl if available) and Binary light curves.
"""

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyArrowPatch
import matplotlib.patches as mpatches

# Try to import VBBinaryLensing for real caustics / magnifications
_vbbl_ok = False
try:
    from VBBinaryLensing import VBBinaryLensing
    vbb = VBBinaryLensing()
    try:
        vbb.SetVerbosity(0)
    except Exception:
        pass
    _vbbl_ok = True
except Exception:
    _vbbl_ok = False

# ---------------------------------------------------------------------------
# Set publication-quality defaults
plt.rcParams['font.size'] = 11
plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.linewidth'] = 1.2

def pspl_magnification(u):
    """PSPL magnification formula"""
    u = np.asarray(u)
    return (u**2 + 2.0) / (u * np.sqrt(u**2 + 4.0))

def source_xy_track(t, t0, tE, u0, alpha_rad):
    """
    Rectilinear motion in Einstein-radius units, lens at origin.
    alpha: angle of source motion (radians), CCW from +x.
    u0: impact parameter at t0.
    """
    tau = (t - t0) / tE
    x = tau * np.cos(alpha_rad) - u0 * np.sin(alpha_rad)
    y = tau * np.sin(alpha_rad) + u0 * np.cos(alpha_rad)
    return x, y

def binary_magnification_vbbl(t, t0, tE, u0, alpha_rad, s, q, rho=0.0):
    """
    Binary magnification using VBBinaryLensing if available.
    Primary at (0,0), companion at (s,0). q = m2/m1.
    """
    if not _vbbl_ok:
        raise RuntimeError("VBBinaryLensing is not available")
    x, y = source_xy_track(t, t0, tE, u0, alpha_rad)
    if hasattr(vbb, "BinaryMag"):
        return np.asarray(vbb.BinaryMag(x, y, s, q, rho))
    elif hasattr(vbb, "mag2"):
        return np.asarray(vbb.mag2(x, y, s, q, rho))
    else:
        return np.asarray([vbb.BinaryMag(xi, yi, s, q, rho) for xi, yi in zip(x, y)])

def binary_magnification_illustrative(t, t0, tE, u0, alpha_rad, s, q):
    """
    Illustration-only fallback when VBBl is not available.
    Approximates a binary curve via a weighted combination of two PSPL lenses.
    NOT physically exact — use VBBl for thesis-quality results.
    """
    x, y = source_xy_track(t, t0, tE, u0, alpha_rad)
    u1 = np.hypot(x, y)
    u2 = np.hypot(x - s, y)
    w1 = 1.0 / (1.0 + q)
    w2 = q / (1.0 + q)
    A1 = pspl_magnification(np.maximum(u1, 1e-6))
    A2 = pspl_magnification(np.maximum(u2, 1e-6))
    return (A1**w1) * (A2**w2)

def safe_binary_magnification(t, t0, tE, u0, alpha_deg, s, q, rho=0.0):
    """Dispatch to VBBl if present, else illustrative fallback."""
    alpha_rad = np.deg2rad(alpha_deg)
    if _vbbl_ok:
        return binary_magnification_vbbl(t, t0, tE, u0, alpha_rad, s, q, rho)
    return binary_magnification_illustrative(t, t0, tE, u0, alpha_rad, s, q)

# =============================================================================
# FIGURE 1: Lens Geometry Schematic
# =============================================================================
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_aspect('equal')
ax.axis('off')

# Observer, Lens, Source positions
obs_x, obs_y = 0, 0
lens_x, lens_y = 5, 0
source_x, source_y = 10, 0

# Draw points
ax.plot(obs_x, obs_y, 'ko', markersize=12, label='Observer (O)')
ax.plot(lens_x, lens_y, 'ro', markersize=12, label='Lens (L)')
ax.plot(source_x, source_y + 0.5, 'b*', markersize=15, label='Source (S)')

# Draw lines
ax.plot([obs_x, lens_x], [obs_y, lens_y], 'k--', linewidth=1.5, alpha=0.7)
ax.plot([lens_x, source_x], [lens_y, source_y + 0.5], 'k--', linewidth=1.5, alpha=0.7)
ax.plot([obs_x, source_x], [obs_y, source_y], 'k:', linewidth=1.5, alpha=0.5)

# Light ray (bent)
t_ = np.linspace(0, 1, 100)
light_x = obs_x + t_ * (source_x - obs_x)
light_y = 0.3 * np.sin(np.pi * t_) + 0.5 * t_
ax.plot(light_x, light_y, 'gold', linewidth=2.5, label='Light Ray')

# Distance annotations
ax.annotate('', xy=(lens_x, -1.5), xytext=(obs_x, -1.5),
            arrowprops=dict(arrowstyle='<->', color='black', lw=1.5))
ax.text(2.5, -1.8, r'$D_\mathrm{l}$', fontsize=14, ha='center')

ax.annotate('', xy=(source_x, -1.5), xytext=(lens_x, -1.5),
            arrowprops=dict(arrowstyle='<->', color='black', lw=1.5))
ax.text(7.5, -1.8, r'$D_\mathrm{ls}$', fontsize=14, ha='center')

# Angle annotations
ax.annotate('', xy=(1.5, 0.3), xytext=(obs_x, obs_y),
            arrowprops=dict(arrowstyle='->', color='red', lw=1.5))
ax.text(1.2, 0.5, r'$\theta$', fontsize=14, color='red')

ax.annotate('', xy=(1.5, 0.05), xytext=(obs_x, obs_y),
            arrowprops=dict(arrowstyle='->', color='blue', lw=1.5))
ax.text(1.8, -0.2, r'$\beta$', fontsize=14, color='blue')

ax.set_xlim([-1, 11])
ax.set_ylim([-3, 3])
ax.legend(loc='upper left', frameon=True, fontsize=11)
plt.tight_layout()
plt.savefig('fig_lens_geometry.pdf', bbox_inches='tight', dpi=300)
plt.close()
print("✅ Created fig_lens_geometry.pdf")

# =============================================================================
# FIGURE 2: PSPL Light Curve with Parameters
# =============================================================================
fig, ax = plt.subplots(figsize=(9, 6))

# Parameters for demonstration
t0 = 50
tE = 20
u0_values = [0.1, 0.3, 0.7]
colors = ['red', 'blue', 'green']

t = np.linspace(0, 100, 500)

for u0_, color in zip(u0_values, colors):
    u_t = np.sqrt(u0_**2 + ((t - t0) / tE)**2)
    A_t = pspl_magnification(u_t)
    ax.plot(t, A_t, color=color, linewidth=2.5, label=f'$u_0 = {u0_}$')

# t0 annotation
ax.axvline(t0, color='black', linestyle=':', linewidth=1.5, alpha=0.7)
ax.text(t0 + 1, 1.5, r'$t_0$', fontsize=14)

# tE annotation
ax.annotate('', xy=(t0 + tE, 2), xytext=(t0, 2),
            arrowprops=dict(arrowstyle='<->', color='black', lw=1.5))
ax.text(t0 + tE/2, 2.2, r'$t_\mathrm{E}$', fontsize=14, ha='center')

ax.set_xlabel('Time [days]', fontsize=13)
ax.set_ylabel('Magnification $A(t)$', fontsize=13)
ax.set_title('PSPL Light Curves for Different Impact Parameters', fontsize=14)
ax.legend(loc='upper right', fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_ylim([1, 10])
plt.tight_layout()
plt.savefig('fig_pspl_lightcurve.pdf', bbox_inches='tight', dpi=300)
plt.close()
print("✅ Created fig_pspl_lightcurve.pdf")

# =============================================================================
# FIGURE 3: Impact Parameter Geometry
# =============================================================================
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_aspect('equal')

# Einstein ring
circle = Circle((0, 0), 1.0, fill=False, edgecolor='black', 
                linewidth=2, linestyle='--', label=r'Einstein Ring ($\theta_E$)')
ax.add_patch(circle)

# Source trajectory
u0_demo = 0.3
t_vals = np.linspace(-2, 2, 50)
x_traj = t_vals
y_traj = u0_demo * np.ones_like(t_vals)
ax.plot(x_traj, y_traj, 'b-', linewidth=2.5, label='Source Path')

# Closest approach point
ax.plot(0, u0_demo, 'ro', markersize=12, label='Closest Approach')
ax.plot(0, 0, 'k*', markersize=18, label='Lens')

# u0 annotation arrow
ax.annotate('', xy=(0, u0_demo), xytext=(0, 0),
            arrowprops=dict(arrowstyle='<->', color='red', lw=2.5))
ax.text(0.15, u0_demo/2, r'$u_0$', fontsize=16, color='red', fontweight='bold')

# Direction arrow
ax.arrow(1.5, u0_demo, 0.3, 0, head_width=0.1, head_length=0.1, 
         fc='blue', ec='blue', linewidth=2)
ax.text(1.9, u0_demo + 0.15, 'Motion', fontsize=12, color='blue')

ax.set_xlim([-2.5, 2.5])
ax.set_ylim([-1.5, 1.5])
ax.set_xlabel(r'$x / \theta_E$', fontsize=13)
ax.set_ylabel(r'$y / \theta_E$', fontsize=13)
ax.set_title('Microlensing Geometry: Impact Parameter', fontsize=14)
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('fig_impact_parameter.pdf', bbox_inches='tight', dpi=300)
plt.close()
print("✅ Created fig_impact_parameter.pdf")

# =============================================================================
# FIGURE 4: Binary Caustics (VBBl if available; otherwise simplified sketch)
# =============================================================================
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_aspect('equal')

s, q = 1.0, 0.01
ax.set_title(rf'Binary Lens Caustics ($s={s}$, $q={q}$)')
ax.set_xlabel(r'$x / \theta_E$')
ax.set_ylabel(r'$y / \theta_E$')
ax.grid(True, alpha=0.3)

def _plot_caustic_segments(ax, segxs, segys, color='r'):
    """Plot one label for the first segment, none for the rest."""
    labeled = False
    for sx, sy in zip(segxs, segys):
        ax.plot(sx, sy, '-', color=color, linewidth=2,
                label='Caustic' if not labeled else None)
        labeled = True

if _vbbl_ok and hasattr(vbb, "Caustic"):
    try:
        cx, cy = vbb.Caustic(s, q)
        # Many VBBl builds return lists of segments; some return single arrays.
        if isinstance(cx, (list, tuple)) and isinstance(cy, (list, tuple)):
            _plot_caustic_segments(ax, cx, cy, color='r')
        else:
            ax.plot(cx, cy, 'r-', linewidth=2, label='Caustic')
    except Exception:
        # Fallback sketch if Caustic call fails
        theta = np.linspace(0, 2*np.pi, 200)
        caustic_x_central = 0.2 * np.cos(theta)
        caustic_y_central = 0.3 * np.sin(theta)
        ax.fill(caustic_x_central, caustic_y_central, color='red', alpha=0.3, label='Central Caustic')
        ax.plot(caustic_x_central, caustic_y_central, 'r-', linewidth=2)

        caustic_x_planet = s + 0.15 * np.cos(theta)
        caustic_y_planet = 0.2 * np.sin(theta)
        ax.fill(caustic_x_planet, caustic_y_planet, color='orange', alpha=0.3, label='Planetary Caustic')
        ax.plot(caustic_x_planet, caustic_y_planet, '-', color='orange', linewidth=2)
else:
    # Original illustrative sketch
    theta = np.linspace(0, 2*np.pi, 200)
    caustic_x_central = 0.2 * np.cos(theta)
    caustic_y_central = 0.3 * np.sin(theta)
    ax.fill(caustic_x_central, caustic_y_central, color='red', alpha=0.3, label='Central Caustic')
    ax.plot(caustic_x_central, caustic_y_central, 'r-', linewidth=2)

    caustic_x_planet = s + 0.15 * np.cos(theta)
    caustic_y_planet = 0.2 * np.sin(theta)
    ax.fill(caustic_x_planet, caustic_y_planet, color='orange', alpha=0.3, label='Planetary Caustic')
    ax.plot(caustic_x_planet, caustic_y_planet, '-', color='orange', linewidth=2)

# Lens positions
ax.plot(0, 0, 'ko', markersize=15, label='Primary Lens')
ax.plot(s, 0, 'ko', markersize=8, label=fr'Companion ($q={q}$)')

# Sample source trajectory
traj_x = np.linspace(-0.5, 1.5, 200)
traj_y = 0.05 * np.ones_like(traj_x)
ax.plot(traj_x, traj_y, 'b--', linewidth=2, alpha=0.7, label='Source Trajectory')

# Separation annotation
ax.annotate('', xy=(s, -0.5), xytext=(0, -0.5),
            arrowprops=dict(arrowstyle='<->', color='black', lw=1.5))
ax.text(s/2, -0.65, rf'$s = {s}\, \theta_E$', fontsize=12, ha='center')

ax.set_xlim([-0.8, 1.8])
ax.set_ylim([-1, 1])
ax.legend(loc='upper left', fontsize=10)
plt.tight_layout()
plt.savefig('fig_binary_caustics.pdf', bbox_inches='tight', dpi=300)
plt.close()
print("✅ Created fig_binary_caustics.pdf")

# =============================================================================
# FIGURE 5: 1D Convolution Example on Light Curve
# =============================================================================
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))

# Generate synthetic PSPL light curve with noise
t = np.linspace(0, 100, 200)
u0 = 0.3
tE = 20
t0 = 50
u_t = np.sqrt(u0**2 + ((t - t0) / tE)**2)
A_t = pspl_magnification(u_t)

# Convert to flux (normalized)
flux = A_t / np.max(A_t)
np.random.seed(0)
flux_noisy = flux + 0.05 * np.random.randn(len(t))

# Plot input
ax1.plot(t, flux_noisy, 'k-', linewidth=1, alpha=0.7, label='Input (Noisy)')
ax1.set_ylabel('Normalized Flux', fontsize=12)
ax1.set_title('Input: Noisy PSPL Light Curve', fontsize=13)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.set_xlim([0, 100])

# Apply 1D convolution with smoothing kernel
kernel_size = 5
kernel = np.ones(kernel_size) / kernel_size  # Simple moving average
flux_smoothed = np.convolve(flux_noisy, kernel, mode='same')

# Plot output
ax2.plot(t, flux_smoothed, 'r-', linewidth=2, label='Output (Smoothed)')
ax2.plot(t, flux, 'b--', linewidth=1.5, alpha=0.5, label='True Signal')
ax2.set_xlabel('Time [days]', fontsize=12)
ax2.set_ylabel('Normalized Flux', fontsize=12)
ax2.set_title(f'Output: After 1D Convolution (Kernel Size = {kernel_size})', fontsize=13)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)
ax2.set_xlim([0, 100])

plt.tight_layout()
plt.savefig('fig_convolution_example.pdf', bbox_inches='tight', dpi=300)
plt.close()
print("✅ Created fig_convolution_example.pdf")

# =============================================================================
# FIGURE 6: CNN Architecture Diagram
# =============================================================================
fig, ax = plt.subplots(figsize=(14, 6))
ax.axis('off')

# Architecture: Input -> Conv1D -> Pool -> Conv1D -> Pool -> Flatten -> Dense -> Output

# Input layer
input_rect = mpatches.FancyBboxPatch((0, 3), 1.5, 2, 
                                      boxstyle="round,pad=0.1", 
                                      edgecolor='black', facecolor='lightblue', linewidth=2)
ax.add_patch(input_rect)
ax.text(0.75, 4, 'Input\n(1500, 1)', ha='center', va='center', fontsize=10, fontweight='bold')

# Conv1D layer 1
conv1_rect = mpatches.FancyBboxPatch((2.5, 2.5), 1.5, 3, 
                                      boxstyle="round,pad=0.1", 
                                      edgecolor='black', facecolor='lightgreen', linewidth=2)
ax.add_patch(conv1_rect)
ax.text(3.25, 4, 'Conv1D\n128 filters\nk=5', ha='center', va='center', fontsize=10, fontweight='bold')

# Arrow
ax.annotate('', xy=(2.5, 4), xytext=(1.5, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Pooling 1
pool1_rect = mpatches.FancyBboxPatch((4.5, 3), 1, 2, 
                                      boxstyle="round,pad=0.1", 
                                      edgecolor='black', facecolor='lightyellow', linewidth=2)
ax.add_patch(pool1_rect)
ax.text(5, 4, 'MaxPool', ha='center', va='center', fontsize=9)

ax.annotate('', xy=(4.5, 4), xytext=(4, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Conv1D layer 2
conv2_rect = mpatches.FancyBboxPatch((6, 2.5), 1.5, 3, 
                                      boxstyle="round,pad=0.1", 
                                      edgecolor='black', facecolor='lightgreen', linewidth=2)
ax.add_patch(conv2_rect)
ax.text(6.75, 4, 'Conv1D\n64 filters\nk=3', ha='center', va='center', fontsize=10, fontweight='bold')

ax.annotate('', xy=(6, 4), xytext=(5.5, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Pooling 2
pool2_rect = mpatches.FancyBboxPatch((8, 3), 1, 2, 
                                      boxstyle="round,pad=0.1", 
                                      edgecolor='black', facecolor='lightyellow', linewidth=2)
ax.add_patch(pool2_rect)
ax.text(8.5, 4, 'MaxPool', ha='center', va='center', fontsize=9)

ax.annotate('', xy=(8, 4), xytext=(7.5, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Flatten
flatten_rect = mpatches.FancyBboxPatch((9.5, 3.2), 1, 1.6, 
                                        boxstyle="round,pad=0.1", 
                                        edgecolor='black', facecolor='lightcoral', linewidth=2)
ax.add_patch(flatten_rect)
ax.text(10, 4, 'Flatten', ha='center', va='center', fontsize=9)

ax.annotate('', xy=(9.5, 4), xytext=(9, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Dense layer
dense_rect = mpatches.FancyBboxPatch((11, 2.5), 1.5, 3, 
                                      boxstyle="round,pad=0.1", 
                                      edgecolor='black', facecolor='plum', linewidth=2)
ax.add_patch(dense_rect)
ax.text(11.75, 4, 'Dense\n128 units\nReLU', ha='center', va='center', fontsize=10, fontweight='bold')

ax.annotate('', xy=(11, 4), xytext=(10.5, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Output
output_rect = mpatches.FancyBboxPatch((13, 3), 1.5, 2, 
                                       boxstyle="round,pad=0.1", 
                                       edgecolor='black', facecolor='lightblue', linewidth=2)
ax.add_patch(output_rect)
ax.text(13.75, 4, 'Output\n2 classes\nSoftmax', ha='center', va='center', fontsize=10, fontweight='bold')

ax.annotate('', xy=(13, 4), xytext=(12.5, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

ax.set_xlim([-0.5, 15])
ax.set_ylim([1.5, 6.5])
ax.set_title('1D CNN Architecture for Microlensing Classification', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('fig_cnn_architecture.pdf', bbox_inches='tight', dpi=300)
plt.close()
print("✅ Created fig_cnn_architecture.pdf")

# =============================================================================
# NEW FIGURE 7: Binary Light Curves for Different (s, q)
# =============================================================================
t = np.linspace(0, 100, 1000)
t0 = 50.0
tE = 20.0
u0 = 0.1        # small impact parameter to show strong features
alpha_deg = 20  # trajectory angle
rho = 0.0       # set >0 for finite-source effects when VBBl is available

pairs = [
    (0.7, 0.5, 'Close, comparable masses'),
    (1.0, 0.1, 'Resonant, lower mass ratio'),
    (1.5, 0.01, 'Wide, planetary regime')
]

fig, ax = plt.subplots(figsize=(9, 6))
for s_i, q_i, label in pairs:
    A = safe_binary_magnification(t, t0, tE, u0, alpha_deg, s_i, q_i, rho=rho)
    ax.plot(t, A, linewidth=2, label=rf'$s={s_i},\ q={q_i}$ — {label}')

# Mark t0
ax.axvline(t0, color='k', linestyle=':', alpha=0.7)
ax.text(t0 + 1, ax.get_ylim()[0] + 0.1*(ax.get_ylim()[1]-ax.get_ylim()[0]), r'$t_0$', fontsize=12)

ax.set_xlabel('Time [days]', fontsize=13)
ax.set_ylabel('Magnification $A(t)$', fontsize=13)
ax.set_title(r'Binary Microlensing Light Curves for Different $(s, q)$', fontsize=14)
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_ylim(bottom=1.0)
plt.tight_layout()
plt.savefig('fig_binary_lightcurves.pdf', bbox_inches='tight', dpi=300)
plt.close()
print("✅ Created fig_binary_lightcurves.pdf")

print("\n" + "="*60)
print("✅ ALL CHAPTER 2 FIGURES (WITH BINARY EVENTS) CREATED SUCCESSFULLY!")
print("="*60)
print("\nCreated files:")
print("  • fig_lens_geometry.pdf")
print("  • fig_pspl_lightcurve.pdf")
print("  • fig_impact_parameter.pdf")
print("  • fig_binary_caustics.pdf  (VBBl if available, else illustrative)")
print("  • fig_convolution_example.pdf")
print("  • fig_cnn_architecture.pdf")
print("  • fig_binary_lightcurves.pdf")
print("\nNow copy these PDFs to your thesis figures directory!")

✅ Created fig_lens_geometry.pdf
✅ Created fig_pspl_lightcurve.pdf
✅ Created fig_impact_parameter.pdf
✅ Created fig_binary_caustics.pdf
✅ Created fig_convolution_example.pdf
✅ Created fig_cnn_architecture.pdf
✅ Created fig_binary_lightcurves.pdf

✅ ALL CHAPTER 2 FIGURES (WITH BINARY EVENTS) CREATED SUCCESSFULLY!

Created files:
  • fig_lens_geometry.pdf
  • fig_pspl_lightcurve.pdf
  • fig_impact_parameter.pdf
  • fig_binary_caustics.pdf  (VBBl if available, else illustrative)
  • fig_convolution_example.pdf
  • fig_cnn_architecture.pdf
  • fig_binary_lightcurves.pdf

Now copy these PDFs to your thesis figures directory!
